# DistributedJets.jl
Package that extends Jets to work with parallel distributed block operators.  This gives us a consistent way to book-keep distributed memory and computation.  It relies heavily on the community (public) DistributedArrays.jl package.

In [1]:
using Distributed

In [2]:
]add DistributedArrays DistributedJets Jets JetPack

   Updating registry at `~/.julia/registries/ChevronETC`
   Updating git-repo `https://chevron@dev.azure.com/chevron/ETC-ESD-PkgRegistry.jl/_git/PkgRegistry.jl`


   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [3]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

Note you need `using Pkg` on each of the workers, accomplished with the `@everywhere` macro

In [4]:
@everywhere using DistributedArrays, DistributedJets, Jets, JetPack

┌ Info: Precompiling DistributedJets [5b65064a-1d0b-5b23-8760-570290dfee43]
└ @ Base loading.jl:1278
┌ Warning: Module JSON with build ID 1204324209965078 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1017
┌ Info: Skipping precompilation since __precompile__(false). Importing DistributedJets [5b65064a-1d0b-5b23-8760-570290dfee43].
└ @ Base loading.jl:1034


We use the same blockop macro as is used in Jets, but now passing in a distributed array rather than an array

In [5]:
A = @blockop DArray(I->[JopDiagonal(rand(2)) for irow in I[1], icol in I[2]], (4,3), workers()[1:4], [2,2])

"Jet linear operator, (6,) → (8,)"

### Where are my blocks
We can use various methods to understand which processes store which blocks

In [6]:
procs(A)

2×2 Array{Int64,2}:
 2  4
 3  5

In [7]:
blockmap(A)

2×2 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:2, 1:2)  (1:2, 3:3)
 (3:4, 1:2)  (3:4, 3:3)

* pid 2 has row-blocks 1:1, and column blocks 1:2
* pid 4 has row-blocks 1:2, and column blocks 3:3
* pid 5 has row-blocks 3:4, and column blocks 1:2
* pid 6 has row-blocks 3:4, and column blocks 3:3

In [8]:
remotecall_fetch(localblockindices, 2, A)

(1:2, 1:2)

In [9]:
remotecall_fetch(localblockindices, 3, A)

(3:4, 1:2)

### Give me my blocks, please

In [10]:
getblock(A,1,1) # fetches block 1,1, and passes a copy of it from pid 2 to the master.

"Jet linear operator, (2,) → (2,)"

In [11]:
remotecall_fetch(getblock, 2, A, 1, 1) # fetch block 1,1 and pass a reference to it on pid 2

"Jet linear operator, (2,) → (2,)"

### distributed block arrays (DBArray)

In [12]:
d = rand(range(A))

8-element DBArray{Float64,Jets.BlockArray{Float64,Array{Float64,1}},Array{Jets.BlockArray{Float64,Array{Float64,1}},1}}:
 0.10427414561097836
 0.9542986168088525
 0.5337628775458072
 0.8239119776309385
 0.025447515266282306
 0.15779078314669426
 0.457226721194963
 0.7364301401056146

In [13]:
procs(d)

2-element Array{Int64,1}:
 2
 3

In [14]:
blockmap(d)

2-element Array{UnitRange{Int64},1}:
 1:2
 3:4

In [15]:
m = rand(domain(A))

6-element DBArray{Float64,Jets.BlockArray{Float64,Array{Float64,1}},Array{Jets.BlockArray{Float64,Array{Float64,1}},1}}:
 0.05117073575263231
 0.9776961866841034
 0.9289492153262939
 0.9321280617637342
 0.7222400494265102
 0.8452462001737115

In [16]:
procs(m)

2-element Array{Int64,1}:
 2
 4

In [17]:
blockmap(m)

2-element Array{UnitRange{Int64},1}:
 1:2
 3:3

In [18]:
getblock(d, 1) # fetch block 1, and passes a copy of it from pid 2 to the master

2-element Array{Float64,1}:
 0.10427414561097836
 0.9542986168088525

In [19]:
setblock!(d, 1, ones(2)) # passes a new array from the master to pid 2, and assigns it to block 1
d

8-element DBArray{Float64,Jets.BlockArray{Float64,Array{Float64,1}},Array{Jets.BlockArray{Float64,Array{Float64,1}},1}}:
 1.0
 1.0
 0.5337628775458072
 0.8239119776309385
 0.025447515266282306
 0.15779078314669426
 0.457226721194963
 0.7364301401056146

In [20]:
remotecall_fetch(getblock, 2, d, 1) # on pid=2 we get a reference to the block

2-element Array{Float64,1}:
 1.0
 1.0

In [21]:
@everywhere function remotegetblock_mutating(d, i)
    dᵢ = getblock(d, i)
    dᵢ .= 2.0
    nothing
end
remotecall_fetch(remotegetblock_mutating, 2, d, 1)
d

8-element DBArray{Float64,Jets.BlockArray{Float64,Array{Float64,1}},Array{Jets.BlockArray{Float64,Array{Float64,1}},1}}:
 2.0
 2.0
 0.5337628775458072
 0.8239119776309385
 0.025447515266282306
 0.15779078314669426
 0.457226721194963
 0.7364301401056146

# Specialized distributed block operators

## tall-and-skinny
Block operators with a single column-block.  This specialization is often used in FWI.  The model is stored on the master.

In [22]:
A = @blockop DArray(I->[JopDiagonal(rand(2)) for irow=1:4, icol=1:1], (4,1))

"Jet linear operator, (2,) → (8,)"

In [23]:
blockmap(A)

4×1 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:1, 1:1)
 (2:2, 1:1)
 (3:3, 1:1)
 (4:4, 1:1)

In [24]:
d = rand(range(A))
blockmap(d)

4-element Array{UnitRange{Int64},1}:
 1:1
 2:2
 3:3
 4:4

In [25]:
m = rand(domain(A))

2-element Array{Float64,1}:
 0.4586095232857317
 0.9579046935343531

### sparse block diagonal
This is the only sparse block operator that we support.  Supporting a larger variety of sparse layouts is possible, but would require an engineering effort to build a proper sparse distributed arrays package.

In [26]:
A = @blockop DArray(
        I->[irow==icol ? JopDiagonal(rand(2)) : JopZeroBlock(JetSpace(Float64,2),JetSpace(Float64,2)) for irow in I[1], icol in I[2]],
        (4,4),
        workers()[1:4],
        [4,1]) isdiag=true

"Jet linear operator, (8,) → (8,)"

In [27]:
procs(A)

4×1 Array{Int64,2}:
 2
 3
 4
 5

In [28]:
blockmap(A)

4×1 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:1, 1:4)
 (2:2, 1:4)
 (3:3, 1:4)
 (4:4, 1:4)

In [29]:
d = rand(range(A))
blockmap(d)

4-element Array{UnitRange{Int64},1}:
 1:1
 2:2
 3:3
 4:4

In [30]:
m = rand(domain(A))
blockmap(m)

4-element Array{UnitRange{Int64},1}:
 1:1
 2:2
 3:3
 4:4